In [1]:
import os

In [2]:
%pwd

'/Users/sameruga/Desktop/learning/nlpprojects/TextSummarizationProject/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/Users/sameruga/Desktop/learning/nlpprojects/TextSummarizationProject'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_dir: Path
    unzip_dir: Path



In [6]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml,create_directories

class ConfigurationManager:
    def __init__(self,
                 config_file_path = CONFIG_FILE_PATH,
                 params_file_path = PARAMS_FILE_PATH
                 ):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)

        create_directories([self.config.artifacts_root])
    

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL= config.source_URL,
            local_data_dir= config.local_data_dir,
            unzip_dir=config.unzip_dir
        )
        return data_ingestion_config


In [7]:
import os
import wget
import zipfile
from textSummarizer.logging import logger
from textSummarizer.utils.common import get_size


In [8]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
    
    def download_file(self):
        path = Path(self.config.local_data_dir)
        if not path.is_file():
            os.chdir(self.config.root_dir)
            filename = wget.download(self.config.source_URL)
            os.chdir("../../")
            logger.info(f"{filename} is downloaded!")            
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_dir))}")
  
    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_dir, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
        

In [9]:
# os.chdir("artifacts/data_ingestion")
# filename = wget.download("https://github.com/entbappy/Branching-tutorial/raw/master/samsumdata.zip")
# print(filename)


In [10]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config = data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2023-09-27 21:15:02,619: INFO: common: yaml file: config/config.yaml loaded successfully]
[2023-09-27 21:15:02,622: INFO: common: yaml file: params.yaml loaded successfully]
[2023-09-27 21:15:02,623: INFO: common: Created Directory at : artifacts ]
[2023-09-27 21:15:02,624: INFO: common: Created Directory at : artifacts/data_ingestion ]
[2023-09-27 21:15:02,626: INFO: 4067379337: File already exists of size: ~ 23073 KB]
